In [1]:
import pickle
import numpy as np

In [2]:
import transformers as ppb

Neither PyTorch nor TensorFlow >= 2.0 have been found.Models won't be available and only tokenizers, configurationand file/data utilities can be used.


In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [25]:
questions = pickle.load(open("questions", "rb"))
questions = list(questions)

In [27]:
while questions.index(''): 
    questions.pop(questions.index(''))

ValueError: '' is not in list

In [29]:
questions.insert(0, 'Um dos nossos usuários está com o e-mail errado no fluig Identity. Como faço para corrigir o endereço de e-mail do usuário nos dois sistemas?')


In [30]:
len(questions)

33544

In [7]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [5]:
from bert_serving.client import BertClient
bc = BertClient(port=5555, port_out=5556)

In [31]:
doc_vecs = bc.encode(questions[:200])

In [32]:
len(doc_vecs)

200

In [33]:
with open('doc_vecs', "bw") as f:
        pickle.dump(doc_vecs, f)

In [34]:
query = 'usuario com email errado no fluig identity'

In [35]:
query_vec = bc.encode([query])[0]

In [44]:
encodes = bc.encode(questions[:200]).fetch_all()

AttributeError: 'numpy.ndarray' object has no attribute 'fetch_all'

In [43]:
type(encodes)

NoneType

In [36]:
# compute normalized dot product as score
score = np.sum(query_vec * doc_vecs, axis=1) / np.linalg.norm(doc_vecs, axis=1)
topk_idx = np.argsort(score)[::-1]

In [37]:
topk_idx

array([ 30, 151,   0,   5,  57,  83,  73,  85, 196,  46,  65,  99,  94,
        81,  59, 178, 170, 150,   4,  78,  64, 164,  52,   1, 169,  88,
       144, 180,  84,  87,  25, 149, 189, 185,  35,  10,  70,  95,   7,
       147, 115, 174, 148, 127, 176, 103,  16,  12,  41, 158,  13,  80,
         8, 191,  74, 119,  49, 114,  23,  50, 116, 194, 153, 112,  48,
        22,  45, 173,  55, 152, 187,  32,  34, 165,  37,  89,  36, 155,
       175, 143, 177,  86, 105, 184, 122, 140,  11,  62, 145, 107,  77,
        51, 146, 167, 136,  92,  28, 183,   2, 104, 198, 126, 118,  76,
       168, 111,  90,  82, 120, 190,  66,  58,  42,  93,  47,  26, 141,
        69, 172,  67, 154, 129, 193,   9, 197,  21, 102, 156, 100,  60,
       138,  98,  96, 123, 101, 199,   6, 162,   3, 139, 110,  79, 171,
       161, 163, 132,  18, 117,  39, 142, 113, 131,  29, 128,  17,  31,
       181, 186,  68, 134, 159,  91,  63, 135,  97,  71, 121, 195, 124,
        15,  24,  38, 192, 109,  20,  72,  33, 179,  27,  14, 12

In [40]:
questions[0]

'Um dos nossos usuários está com o e-mail errado no fluig Identity. Como faço para corrigir o endereço de e-mail do usuário nos dois sistemas?'

In [19]:
questions[4]

'Um dos nossos usuários está cadastrado no fluig Identity e no fluig com o endereço de e-mail incorreto.Como faço para corrigir o endereço de e-mail do usuário nos dois sistemas?'

In [38]:
questions[30]

'Ao acessar a plataforma com o usuário wcmadmin, o fluig não está habilitado o campo para assinatura digital. Aparece a mensagem: "Módulo de Assinatura Digital não está habilitado. Entre em contato com o administrador."'

In [39]:
questions[151]

'É possível configurar o banco de dados MS SQL Server para acessar o fluig ESB diretamente sem a necessidade de iniciar pelo default Derby?'

#### Importing the pre trained model

In [13]:
import warnings
warnings.filterwarnings('ignore')

#model_class, tokenizer_class, pretrained_weights = (ppb.BertForSequenceClassification, ppb.BertTokenizer, 'bert-base-multilingual-cased')
tokenizer_class, pretrained_weights = (ppb.BertTokenizer, 'bert-base-multilingual-cased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
#model = model_class.from_pretrained(pretrained_weights,
#                                    num_labels = 9, # integer, default 2. Number of classes to use when the model is a classification model (sequences/tokens)
#                                    output_attentions = False, # Should the model returns attentions weights.
#                                    output_hidden_states = False,  # Should the model returns all hidden-states.
#                                   )

#### Tokenization

In [14]:
tokenized = [tokenizer.encode(x, add_special_tokens=True) for x in questions[:5]]

In [16]:
# Print the original sentence.
print(' Original: ', questions[1])

# Print the sentence mapped to token ids.
print('Token IDs: ', ' '.join(tokenizer.convert_ids_to_tokens(tokenized[1])))

# Print the sentence split into tokens.
print('Tokenized: ', tokenized[1])

 Original:  Ao tentar alterar as informações de um usuário e bloquear o mesmo, uma mensagem é apresentada: result returns more than one elements. Por que essa mensagem aparece?
Token IDs:  [CLS] Ao tentar alter ##ar as informações de um usu ##ário e bloque ##ar o mesmo , uma mens ##agem é apresenta ##da : result returns more than one elements . Por que essa mens ##agem aparece ? [SEP]
Tokenized:  [101, 17607, 110694, 42141, 10354, 10146, 68263, 10104, 10293, 44224, 15211, 173, 99406, 10354, 183, 13544, 117, 10437, 12380, 29111, 263, 49320, 10229, 131, 14493, 38302, 10798, 11084, 10464, 17464, 119, 12399, 10121, 21553, 12380, 29111, 20962, 136, 102]


#### Padding

Pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).

In [20]:
max_len = 0
for i in tokenized:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized])

In [21]:
np.array(padded).shape

(5, 243)

In [22]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(5, 243)

In [23]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

NameError: name 'torch' is not defined

In [48]:
input_ids.shape

torch.Size([70159, 512])

In [49]:
attention_mask.shape

torch.Size([70159, 512])

In [ ]:
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()